# BAB 1a, Data Collecting: Daily Charts Scrapping
Sumber data: https://kworb.net/spotify/country/id_daily.html

Hari/Tanggal: Selasa/20 Juni 2023

In [ ]:
!pip install spotipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 6.1 MB/s eta 0:00:00


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import datetime
import calendar

In [ ]:
def get_today_date():
    # Mendapatkan tanggal hari ini
    today = datetime.date.today()

    # Mendapatkan nama bulan dalam bahasa Indonesia
    month_name = calendar.month_name[today.month]

    # Mengubah format tanggal menjadi string dalam bentuk "DD MonthName YYYY"
    formatted_date = f"{today.day} {month_name} {today.year}"

    return formatted_date

In [ ]:
def create_spotify_playlist(dataframe, playlist_name, client_id, client_secret):
    # Menginisialisasi koneksi dengan Spotify API menggunakan Spotipy
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id, client_secret=client_secret, redirect_uri='https://example.com/callback', scope='playlist-modify-private'))

    # Membuat playlist baru
    playlist = sp.user_playlist_create(sp.current_user()['id'], playlist_name, public=False)

    # Menambahkan lagu ke playlist berdasarkan kolom 'Artist' dan 'Title' dalam DataFrame
    for index, row in dataframe.iterrows():
        artist = row['Artist']
        title = row['Title']

        # Mencari lagu berdasarkan artist dan title
        results = sp.search(q=f'artist:{artist} track:{title}', type='track')

        # Memastikan ada hasil pencarian lagu
        if results['tracks']['total'] > 0:
            track_uri = results['tracks']['items'][0]['uri']

            # Menambahkan lagu ke playlist
            sp.playlist_add_items(playlist['id'], [track_uri])

    # Mengembalikan URL playlist yang dibuat
    return playlist['external_urls']['spotify']


In [ ]:
today_date = get_today_date()
playlist_name = f"200 Lagu Teratas di Indonesia - {today_date}"
client_id = '3ff80dfa932f42ccba937689112ebaf8'
client_secret = 'b222a7c17fda476aa56057b10f263a7d'

In [ ]:
# URL untuk mengakses tabel Spotify
url = "https://kworb.net/spotify/country/id_daily.html"

# Membaca tabel dari URL menggunakan pandas
data = pd.read_html(url)

# Mengambil tabel pertama dari hasil pembacaan (indeks 0)
df = data[0]

# Mengganti nama kolom 'Artist and Title' menjadi 'Artist'
df.rename(columns={'Artist and Title': 'Artist'}, inplace=True)

# Membagi kolom 'Artist' menjadi 'Artist' dan 'Title'
df[['Artist', 'Title']] = df['Artist'].str.split(' - ', 1, expand=True)


<ipython-input-15-c6be206143c2>:14: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df[['Artist', 'Title']] = df['Artist'].str.split(' - ', 1, expand=True)


In [ ]:
from IPython.display import display, HTML

# URL untuk mendapatkan authorization code
auth_url = "https://accounts.spotify.com/authorize?client_id=3ff80dfa932f42ccba937689112ebaf8&response_type=code&redirect_uri=https://example.com/callback&scope=playlist-modify-private"

# Menampilkan URL sebagai tautan yang dapat diklik
display(HTML(f"Click <a href='{auth_url}' target='_blank'>here</a> to authorize access to your Spotify account."))

# Meminta pengguna untuk memasukkan authorization code secara manual
authorization_code = input("Enter the authorization code: ")

In [ ]:
# Membuat playlist di Spotify
playlist_url = create_spotify_playlist(df, playlist_name, client_id, client_secret)
print(f"Playlist '{playlist_name}' berhasil dibuat di Spotify. URL playlist: {playlist_url}")

Enter the URL you were redirected to: https://example.com/callback?code=AQAYBtJXD-HWF4NrvliW-Ma5cEz321dX6ZHwjqGB-ULb4ZfpH_s9uZkJVMNbbmOE518Ew17QTNbxhjpykp3-B-oOX2L8aSX3UzCYslgO9V_VlAANS821sKMI-_c-_dosOTtFNl_i2wyw9DacatL1PQxwh8iv1nPBHLbYc1mpwdeWtcbjGrg1lTdqjvm6Vqk9YVGkt320p1tv
Playlist '200 Lagu Teratas di Indonesia - 20 June 2023' berhasil dibuat di Spotify. URL playlist: https://open.spotify.com/playlist/6uIBikeo5OaUKmLBOV1AyG


In [ ]:
df

,Pos,P+,Artist,Days,Pk,(x?),Streams,Streams+,7Day,7Day+,Total,Title
0,1,=,Anggi Marito,165,1,(x52),1365488,-58353.0,9990017,-91535,121904719,Tak Segampang Itu
1,2,=,Raim Laode,127,1,(x64),915014,-32150.0,6559272,-75064,144877159,Komang
2,3,=,Mahalini,142,1,(x19),837816,-29380.0,6168922,-74735,139560654,Sial
3,4,=,Miley Cyrus,96,3,(x10),818843,-22035.0,5728587,36228,61638287,Angels Like You
4,5,=,Alan Walker,63,4,NaN,710355,-25291.0,5148833,-72208,31865235,Not You (w/ Emma Steinbakken)
...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,-10,ENHYPEN,28,78,NaN,108287,-6274.0,836847,-24269,4057500,Bite Me
196,197,-5,The Rain,376,102,NaN,107513,-4073.0,811493,-10436,25784174,Terlatih Patah Hati
197,198,+2,Fabio Asher,478,2,(x19),107303,-1318.0,669439,-7114,107266155,Bertahan Terluka
198,199,-4,Sheila On 7,1160,40,NaN,106831,-3220.0,593009,-24501,37241666,Seberapa Pantas


In [ ]:
df.to_csv('df_daily.csv')